# 02 - Data from the Web

In [5]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

 We will first download all the data needed to perform the analysis for Master students.
 
 Then we will calculate time spent at EPFL for each student.
 
 Finally we will calculate if average time for students having particural Specialization is significantly different.


We start by downloading the form from isa webpage.
Using console in webbrowser we found a direct link to the frame containing the form. 

In [6]:

urlParams = {'ww_i_reportModel': '133685247'}
isa_baseurl = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS'
isa_formurl= isa_baseurl + ".filter"
r = requests.get(isa_formurl, params=urlParams)
r.url + " " + str(r.status_code)

'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247 200'

We are building a framework that will let us easily get a pandas Dataframe if we provide parameters we see on the isa webpage.

To achieve it, we implement a function that accepts as argument label of input from ISA webpage (for instance "Unité académique") and label of option (for instance "Informatique") and returns key-value pair that should be included as GET parameter.

In [7]:
soup = BeautifulSoup(r.text, 'html.parser')
form = form = soup.find('form')

def findSelectNameAndOptionValue(inputLabel, optionLabel):
    th = form.find('th', string=inputLabel)
    td = th.nextSibling
    select = td.find('select')
    option = select.find('option', string=optionLabel)
    return (select['name'], option['value'])

findSelectNameAndOptionValue("Unité académique", "Informatique")

('ww_x_UNITE_ACAD', '249847')

Now we can build a function that generates all the parameters for GET request.
Some parameters are always the same and we know them because we used POSTMAN.
We decided to download data as HTML.

In [8]:
def generateParams(humanReadableParams):
    fixedParams = {
        'ww_i_reportModel': '133685247',
        'ww_i_reportModelXsl': '133685270',
        'ww_x_GPS': '-1',
    }
    params = dict([findSelectNameAndOptionValue(x, y) for (x, y) in humanReadableParams.items()])
    return {**fixedParams, **params}

# Example
exampleParams = {
    "Unité académique": "Informatique",
    "Période académique": "2016-2017",
    "Période pédagogique": "Bachelor semestre 1",
    "Type de semestre": "Semestre d'automne",
}
generateParams(exampleParams)

{'ww_i_reportModel': '133685247',
 'ww_i_reportModelXsl': '133685270',
 'ww_x_GPS': '-1',
 'ww_x_HIVERETE': '2936286',
 'ww_x_PERIODE_ACAD': '355925344',
 'ww_x_PERIODE_PEDAGO': '249108',
 'ww_x_UNITE_ACAD': '249847'}

The data for given semester and year always comes as html table.
For `Bachelor semestre`, `Master semestre`, `Projet Master` the structure of the table is always the same,
so we can parse it all with one function.

In [9]:
def extractTable(html):
    """Extract the table from HTML into a pandas Dataframe."""
    BStable = BeautifulSoup(html, 'html.parser').find('table')
    BSRows = BStable.findAll('tr')
    # BSrows[0] is just the title of the table, so we ignore it.
    headerBSRow = BSRows[1]
    headers = [th.text for th in headerBSRow]
    # We drop the last column, because it's always empty (originally it is used to make the HTML table look better).
    rows = [[tr.text for tr in BSRow.findAll('td')[:-1]] for BSRow in BSRows[2:]]
    return pd.DataFrame(data=rows, columns=headers)

Now we combine generating parameters and parsing and receive the promised function.

In [10]:
def getDataFrame(humanReadableParams):
    isa_dataurl = isa_baseurl + ".html"
    r = requests.get(isa_dataurl, params=generateParams(humanReadableParams))
    return extractTable(r.text)

getDataFrame({
    "Unité académique": "Informatique",
    "Période académique": "2016-2017",
    "Période pédagogique": "Bachelor semestre 1",
    "Type de semestre": "Semestre d'automne",
}).head()

,Civilité,Nom Prénom,Orientation Bachelor,Orientation Master,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,No Sciper
0,Monsieur,Abbey Alexandre,,,,,,Présent,,,235688
1,Monsieur,Ahn Seongho,,,,,,Présent,,,274015
2,Madame,Alemanno Sara,,,,,,Présent,,,268410
3,Monsieur,Althaus Luca,,,,,,Présent,,,271464
4,Monsieur,Assi Karim,,,,,,Présent,,,274518


Before actually downloading the data it's worth considering what we will need.
By simply clicking things on ISA webpage we noted that data for `Projet Master` is really scarce.
The same applies to `Master semestre 4`. Therefore we decided not to use it.

We want to estimate only the time student spends at EPFL on regular studying (no Projet Master, no internship).
Still however, we hope to exclude the students who didn't graduate. How can we guess if student graduated?

Well, we use 2 criteria:
1. Every student should at least appear in `Master semestre 1` and `Master semestre 2`.
2. If a student does a `Mineur` he must appear in `Master semestre 3` because it's too many ECTS to do in 2 semesters.

These criteria will also exclude some students who started before 2007, but it only makes the sample smaller.

Finally, the time spent at EPFL is calculated in semesters as the difference between chronogically first and last semesters in which student was registered. This account for all weird orders in which student could pass the semesters.

Instead of remembering years and whether semester was in the first part of the year or in the second, we simply order all semester chronogically starting from 0.


We're downloading all the tables from ISA webpage. With each table we remember what type of semester (master 1, 2, 3) and which semester chronogically it describes.

In [12]:
def getMasterData():
    """Get all needed Master data as list of tuples (semesterType, semesterNo, dataframe)."""
    startYear = 2007
    endYear = 2016
    data = []
    print("Running... ", end='')
    for year in range (startYear, endYear):
        period = "{:d}-{:d}".format(year, year + 1)
        m1Params = {
            "Unité académique": "Informatique",
            "Période académique": period,
            "Période pédagogique": "Master semestre 1",
            "Type de semestre": "Semestre d'automne",
        }
        m2Params = {
            "Unité académique": "Informatique",
            "Période académique": period,
            "Période pédagogique": "Master semestre 2",
            "Type de semestre": "Semestre de printemps",
        }
        m3Params = {
            "Unité académique": "Informatique",
            "Période académique": period,
            "Période pédagogique": "Master semestre 3",
            "Type de semestre": "Semestre d'automne",
        }
        m1 = (1, (year - startYear) * 2    , getDataFrame(m1Params)) 
        m2 = (2, (year - startYear) * 2 + 1, getDataFrame(m2Params))
        m3 = (3, (year - startYear) * 2    , getDataFrame(m3Params))
        data.extend([m1, m2, m3])
        print(">> " + str(year) + ' ', end='')
    print("...finished!")
    return data
        
masterData = getMasterData()

Running... >> 2007 >> 2008 >> 2009 >> 2010 >> 2011 >> 2012 >> 2013 >> 2014 >> 2015 ...finished!


In [13]:
# Let's check we get sensible number of rows everywhere.

def checkData(data):
    return [(semType, semNo, dataFrame.shape) for (semType, semNo, dataFrame) in data]

checkData(masterData)

[(1, 0, (71, 11)),
 (2, 1, (87, 11)),
 (3, 0, (65, 11)),
 (1, 2, (60, 11)),
 (2, 3, (64, 11)),
 (3, 2, (48, 11)),
 (1, 4, (52, 11)),
 (2, 5, (62, 11)),
 (3, 4, (34, 11)),
 (1, 6, (96, 11)),
 (2, 7, (109, 11)),
 (3, 6, (40, 11)),
 (1, 8, (102, 11)),
 (2, 9, (123, 11)),
 (3, 8, (67, 11)),
 (1, 10, (88, 11)),
 (2, 11, (130, 11)),
 (3, 10, (87, 11)),
 (1, 12, (104, 11)),
 (2, 13, (138, 11)),
 (3, 12, (75, 11)),
 (1, 14, (104, 11)),
 (2, 15, (151, 11)),
 (3, 14, (81, 11)),
 (1, 16, (132, 11)),
 (2, 17, (196, 11)),
 (3, 16, (92, 11))]

To make the further transformations easier, we stack up all the dataframes and metainformation together into one Dataframe. The structure of the dataframe is rather self-explaining. Note that for now there may be multiple multiple rows related to one student.

In [16]:
def prepare(triple):
    (semType, semNo, dataFrame) = triple
    df = dataFrame[['No Sciper', 'Spécialisation']].copy()
    df['Mineur'] = dataFrame['Mineur'] != ''
    df['master1'] = semType == 1
    df['master2'] = semType == 2
    df['master3'] = semType == 3
    df['semNo'] = semNo
    return df

concatedData = pd.concat(map(prepare, masterData))
concatedData.sample(5)

,No Sciper,Spécialisation,Mineur,master1,master2,master3,semNo
59,215127,,False,True,False,False,8
60,186250,Foundations of Software,False,False,True,False,9
69,211018,Internet computing,False,False,True,False,9
93,227660,,False,True,False,False,16
12,184991,"Signals, Images and Interfaces",False,False,False,True,10


Let's see how many unique students we consider.

In [17]:
len(concatedData['No Sciper'].unique())

855

We are aggregating the data for each student to be able to tell whether he ever was in master 1, 2, 3 or has a Mineur and last but not least how long he was at EPFL.
We'll treat the Specialisations individually, because as it happens some students may have more than one.

In [18]:
def aggregateData(data):
    neededData = data[['No Sciper', 'Mineur', 'master1', 'master2', 'master3', 'semNo']]
    aggregations = {
        'Mineur': 'any',
        'master1': 'any',
        'master2': 'any',
        'master3': 'any',
        'semNo': lambda x: max(x) - min(x) + 1
    }
    return data.groupby(['No Sciper'], as_index=False).agg(aggregations)
studentsData = aggregateData(concatedData)
studentsData.head()

,No Sciper,Mineur,master3,semNo,master1,master2
0,128911,False,True,2,False,True
1,129093,False,True,2,False,True
2,129326,False,True,2,False,True
3,138088,False,False,1,True,False
4,145546,False,True,1,False,False


It lets us choose "good" students, that is the students that, we believe, graduated.

In [20]:
goodStudents = studentsData \
    .query('master1 and master2 and (master3 or not Mineur)') \
    [['No Sciper', 'semNo']] \
    .reset_index(drop=True)
    
print(len(goodStudents))
goodStudents.head()

743


,No Sciper,semNo
0,146330,3
1,146742,9
2,146929,2
3,147008,4
4,152232,3


Let's see what specialisations exist.

In [21]:
specials = concatedData \
    .query("Spécialisation != ''") \
    ['Spécialisation'] \
    .unique().tolist()
specials

['Internet computing',
 'Foundations of Software',
 'Biocomputing',
 'Signals, Images and Interfaces',
 'Computer Engineering - SP',
 'Service science',
 'Software Systems',
 'Information Security - SP',
 'Data Analytics',
 'Internet Information Systems',
 'Computer Science Theory']

And we can choose the students pursuing a particular specialisation...

In [25]:
def findSpecialStudents(special):
    specialScipers = concatedData \
        .query("Spécialisation == @special") \
        [['No Sciper']] \
        .drop_duplicates() \
        .reset_index(drop=True)
    return specialScipers.merge(goodStudents, on='No Sciper')

specialStudents = {spec: findSpecialStudents(spec) for spec in specials}

print(len(specialStudents['Biocomputing']))
specialStudents['Biocomputing']

4


,No Sciper,semNo
0,172611,3
1,154573,5
2,169734,5
3,183185,3


Results, finally!

#### Average time spent at EPFL in semesters

In [27]:
print(">> All students <<")
print("Average: {:f}".format(goodStudents['semNo'].mean()))
print("Number of students: {:d}".format(len(goodStudents)))

>> All students <<
Average: 3.107672
Number of students: 743


In [43]:
means = [df['semNo'].mean() for df in specialStudents.values()]
studentNums = [len(df) for df in specialStudents.values()]

pd.DataFrame({
    'Spécialisation': list(specialStudents.keys()),
    'mean in semesters': means,
    'number of students': studentNums,
    })

,Spécialisation,mean in semesters,number of students
0,Service science,3.600000,5
1,"Signals, Images and Interfaces",4.000000,31
2,Software Systems,2.941176,17
3,Computer Engineering - SP,3.300000,20
4,Data Analytics,2.000000,4
5,Information Security - SP,2.571429,7
6,Computer Science Theory,2.000000,1
7,Internet Information Systems,2.000000,1
8,Foundations of Software,3.725806,62
9,Internet computing,3.623656,93


We shouldn't treat the Spécialisations with 1 person seriously, but we can finish the analysis.
We use 2-sample t-test analysis, because it checks if two populations have the same average.
However the populations should have empty intersection, so for each specialisation we compare it with all the __other__ students.

#### Is the average time spent at EPFL significantly different?

In [45]:
import scipy.stats
def pvalue(spec):
    chosenStudents = specialStudents[spec]
    indexes = goodStudents['No Sciper'].isin(chosenStudents['No Sciper'])
    otherStudents = goodStudents[~indexes]
    return scipy.stats.ttest_ind(
        a=chosenStudents['semNo'],
        b=otherStudents['semNo'],
        equal_var=False).pvalue

result = [pvalue(spec) <= 0.05 for spec in specials]
pd.DataFrame({
    'Spécialisation': specials,
    'is average significantly different': result,
    })

/Users/nutka/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:82: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


,Spécialisation,is average significantly different
0,Internet computing,True
1,Foundations of Software,True
2,Biocomputing,False
3,"Signals, Images and Interfaces",True
4,Computer Engineering - SP,False
5,Service science,False
6,Software Systems,False
7,Information Security - SP,False
8,Data Analytics,True
9,Internet Information Systems,False
